# Assignment #3: A simple language classifier with scikit-learn and PyTorch

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired and simplified from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks with sklearn and PyTorch
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [4]:
import bz2
import json
import os
import numpy as np
import requests
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import hashlib
from tqdm import tqdm

In [5]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

## Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider six languages only: French (fra), Japanese (jpn), Chinese (cmn), English (eng), Swedish (swe), and Danish (dan). Below is an excerpt of the Tatoeba dataset limited to three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår. 
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 23, 2021.

### Understanding the ${X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified ${X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will __include this matrix in your report__. 

$\mathbf{X} =
\begin{bmatrix}
0& 0& 1& 0& 0& 0\\
1& 0& 0& 0&0& 0\\
3& 1& 2& 1&0& 0\\
8& 0& 8& 1&0&0\\
1& 0& 1& 0&0& 0\\
4& 1& 6& 1&0& 0\\
4& 0& 1& 1&0& 0\\
5& 2& 2& 0&1& 0\\
2& 0& 2& 1&0& 0\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

To help you check your counts, you can use the `str.count()` function

In [6]:
example_ngrams = ['a', 'b', 'n', 'an', 'ba', 'na']

In [7]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('a')

8

In [8]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('an')

1

In [9]:
my_string = """Vi hade roligt på stranden igår."""
row = []
for ngram in example_ngrams:
    row += [my_string.count(ngram)]
row

[2, 0, 2, 1, 0, 0]

## Getting the Dataset

Before you start programming, download the Tatoeba dataset. You can use the instructions:

In [10]:
#!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2023-09-25 10:26:12--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Slår upp downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194


Ansluter till downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443 … ansluten.
HTTP-begäran skickad, väntar på svar... 200 OK
Längd: 183539203 (175M) [application/octet-stream]
Sparar till: ”sentences.tar.bz2”

sentences.tar.bz2   100%[===================>] 175,04M  32,4MB/s    om 5,5s    

2023-09-25 10:26:17 (31,7 MB/s) - ”sentences.tar.bz2” sparades [183539203/183539203]



In [11]:
#!tar -xvjf sentences.tar.bz2

x sentences.csv

### Loading the Dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [12]:
dataset_large = open('sentences.csv', encoding='utf8').read().strip()
dataset_large = dataset_large.split('\n')
dataset_large[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

The size may vary as new documents are added every day to _Tatoeba_

In [13]:
len(dataset_large)

11626127

Run the code to split the fields and remove possible whitespaces

In [14]:
dataset_large = list(map(lambda x: tuple(x.split('\t')), dataset_large))
dataset_large = list(map(lambda x: tuple(map(str.strip, x)), dataset_large))
dataset_large[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

In [15]:
from collections import Counter
counter = Counter(map(lambda x: x[1], dataset_large))

Again the figures may vary

In [16]:
counter.most_common(30)

[('eng', 1832557),
 ('rus', 1015379),
 ('ita', 863210),
 ('tur', 729325),
 ('epo', 728733),
 ('kab', 686765),
 ('ber', 650115),
 ('deu', 627037),
 ('fra', 569043),
 ('por', 422236),
 ('spa', 402610),
 ('hun', 394834),
 ('jpn', 238601),
 ('heb', 200845),
 ('ukr', 184423),
 ('nld', 177097),
 ('fin', 146735),
 ('pol', 123050),
 ('lit', 93949),
 ('mkd', 78095),
 ('tgl', 74924),
 ('ces', 73650),
 ('cmn', 73646),
 ('mar', 73277),
 ('ara', 62803),
 ('dan', 59011),
 ('tok', 54067),
 ('swe', 53199),
 ('lat', 49211),
 ('srp', 47878)]

## Restricting the Dataset to a few Languages 

The Tatoeba dataset is very large. You will first extract a subset of it

Write the code to extract texts in the languages below. For each language, you limit the number of documents to 50,000 or less if the language has less documents.
You will call the resulting dataset: `dataset`

The languages

In [17]:
langs = ['fra', 'cmn', 'jpn', 'eng', 'swe', 'dan']

The maximal number of documents per language

In [18]:
MAX_DOCS = 50000

Write a loop that:
1. Extracts a list of all the documents in a certain language from the dataset
2. Shuffles this list with `random.shuffle()`
3. Adds `MAX_DOCS` to `dataset`. You just need to use a slice

In [39]:
# Write your code here
dataset = []
for lang in langs:
    dataset_Lang = list(filter(lambda x: x[1] == lang, dataset_large))
    random.shuffle(dataset_Lang)
    dataset_Lang = dataset_Lang[:MAX_DOCS]
    dataset += dataset_Lang

In [43]:
random.shuffle(dataset)

In [44]:
len(dataset)

300000

In [45]:
dataset[:5]

[('2478946', 'dan', 'Min bror taler for meget.'),
 ('1999574', 'dan', 'Han er meget nysgerrig.'),
 ('2394580', 'cmn', '体育无国界。'),
 ('4024010', 'fra', 'Cela a un goût désagréable.'),
 ('11358498', 'swe', 'Jag har ett sparkonto.')]

## Utilities

Before you can use the dataset to train a model, you need to convert it into numbers. You will carry this with out the following steps and you will write a corresponding function.
1. You will extract the $n$-grams up to trigrams (`all_ngrams()`);
2. Trigrams can create many symbols that most student's machines cannot process. You will reduce their numbers using hash codes (`hash_ngrams()`);
3. You will compute the relative frequencies of the $n$-grams, replaced here by the hash codes (`calc_ref_freq()`).
4. The results will be stored in three dictionaries, for characters, bigrams, and trigrams. You will merge these dictionaries into one (`shift_keys()`).

You will then apply the functions to vectorize the dataset.

### Extracting $n$-grams
The goal of this section is that you extract the $n$-grams from a text. By default, you will lowercase the text. The result will have the form: `[chars, bigrams, trigrams]`

Write a function to extract the $n$-grams of a sentence: `ngrams(sentence, n=1, lc=True)`, `n` is a parameters. You can use list slices for this.

In [48]:
# Write your code here
def ngrams(sentence, n=1, lc=True):
    ngram_l = []
    if lc:
        sentence = sentence.lower()
    for i in range(len(sentence)-n+1):
        ngram_l.append(sentence[i:i+n])
    return ngram_l

In [49]:
ngrams('try something.')

['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.']

In [50]:
ngrams('try something.', n=2)

['tr', 'ry', 'y ', ' s', 'so', 'om', 'me', 'et', 'th', 'hi', 'in', 'ng', 'g.']

We now use this function to extract all the $n$-grams

In [51]:
def all_ngrams(sentence, max_ngram=3, lc=True):
    all_ngram_list = []
    for i in range(1, max_ngram + 1):
        all_ngram_list += [ngrams(sentence, n=i, lc=lc)]
    return all_ngram_list

In [52]:
all_ngrams('try something.')

[['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.'],
 ['tr',
  'ry',
  'y ',
  ' s',
  'so',
  'om',
  'me',
  'et',
  'th',
  'hi',
  'in',
  'ng',
  'g.'],
 ['try',
  'ry ',
  'y s',
  ' so',
  'som',
  'ome',
  'met',
  'eth',
  'thi',
  'hin',
  'ing',
  'ng.']]

### Hashing

We consider languages with many characters that will make the number of bigrams and trigrams impossible to process. We will use the _hashing trick_ to reduce them, where we will gather $n$-grams into subsets using hash codes.

Each item will have this format:
`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

#### Description

Python has a built-in hashing function that returns a unique numerical signature for a given string

In [53]:
hash('a'), hash('ab'), hash('abc')

(-695212193385185405, 8999113146442834582, -101788869977028125)

If we take the remainder (modulo) of a division by 5, we reduce the possible codes to: 0, 1, 2, 3, or 4

In [54]:
list(map(lambda x: x % 5, (hash('a'), hash('ab'), hash('abc'))))

[0, 2, 0]

#### Implementation

We set maximal numbers for our $n$-grams using these divisors

In [55]:
MAX_CHARS = 521
MAX_BIGRAMS = 1031
MAX_TRIGRAMS = 1031

Here strings have integer codes within the range [0, `MAX_CHARS`[

In [56]:
list(map(lambda x: x % MAX_CHARS, (hash('a'), hash('ab'), hash('abc'))))

[490, 151, 360]

Hash codes may vary across machines and Marcus Klang wrote this function to have reproducible codes

In [57]:
def reproducible_hash(string):
    """
    reproducible hash on any string
    
    Arguments:
       string: python string object
    
    Returns:
       signed int64
    """
    
    # We are using MD5 for speed not security.
    h = hashlib.md5(string.encode("utf-8"), usedforsecurity=False)
    return int.from_bytes(h.digest()[0:8], 'big', signed=True)

In [58]:
reproducible_hash('a')

919145239626757800

In [59]:
reproducible_hash('a') % MAX_CHARS

234

### Converting $n$-grams to hash codes
You will now convert the $n$-grams to hash codes


In [60]:
MAXES = [MAX_CHARS, MAX_BIGRAMS, MAX_TRIGRAMS]

Create a `hash_ngrams` function that creates a list of hash codes from a list of $n$-grams. As arguments, you will have the list of $n$-grams `[chars, bigrams, trigrams]` as well as the list of dividers (`MAXES`).

The output format will be a list of three lists:

`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

In [64]:
# Write your code
def hash_ngrams(ngrams, modulos):
    hash_codes = []
    for i in range(len(ngrams)):
        hash_codes += [list(map(lambda x: x % modulos[i], map(reproducible_hash, ngrams[i])))]
    return hash_codes

In [65]:
all_ngrams('try something.')

[['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.'],
 ['tr',
  'ry',
  'y ',
  ' s',
  'so',
  'om',
  'me',
  'et',
  'th',
  'hi',
  'in',
  'ng',
  'g.'],
 ['try',
  'ry ',
  'y s',
  ' so',
  'som',
  'ome',
  'met',
  'eth',
  'thi',
  'hin',
  'ing',
  'ng.']]

In [66]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

### Functions to Count Hash Codes

Write a function `calc_rel_freq(codes)` to count the codes. As in CLD3, you will return the relative frequencies.

This is just an application of `Counter` to a list of codes and then a division by the length.

The input is a list of codes and the output is a `Counter` object of relative frequencies.

In [112]:
# Write your code
def calc_rel_freq(codes):
    count = Counter(codes)
    return Counter({k: v / len(codes) for k, v in count.items()})
    #return cnt

In [113]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

In [114]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

[Counter({432: 0.14285714285714285,
          437: 0.07142857142857142,
          309: 0.07142857142857142,
          86: 0.07142857142857142,
          331: 0.07142857142857142,
          97: 0.07142857142857142,
          100: 0.07142857142857142,
          32: 0.07142857142857142,
          332: 0.07142857142857142,
          233: 0.07142857142857142,
          310: 0.07142857142857142,
          31: 0.07142857142857142,
          442: 0.07142857142857142}),
 Counter({6: 0.07692307692307693,
          765: 0.07692307692307693,
          224: 0.07692307692307693,
          203: 0.07692307692307693,
          557: 0.07692307692307693,
          176: 0.07692307692307693,
          590: 0.07692307692307693,
          711: 0.07692307692307693,
          527: 0.07692307692307693,
          757: 0.07692307692307693,
          919: 0.07692307692307693,
          57: 0.07692307692307693,
          685: 0.07692307692307693}),
 Counter({848: 0.08333333333333333,
          617: 0.08333333333333

### Merge the Dictionaries

In the results above, we have three counter objects with numerical keys (the hash codes). You will build one dictionary of them.

There is a key overlap and we must take care that a same hash code for the unigrams is not the same as in the bigrams. We will then shift the keys.

The keys range from:
1. Unigrams from 0 to 521, [0, MAX_CHARS[
2. Bigrams from 0 to 1031, [0, MAX_BIGRAMS[
3. Trigrams from 1 to 1031, [0, MAX_TRIGRAMS[

You will leave the unigrams keys as they are. You will shift the bigram keys by MAX_CHARS, and the trigram keys by MAX_CHARS + MAX_BIGRAMS. You can reuse the code below

In [119]:
MAX_SHIFT = []
for i in range(len(MAXES)):
    MAX_SHIFT += [sum(MAXES[:i])]

In [120]:
MAX_SHIFT

[0, 521, 1552]

Write a `shift_keys(dicts, MAX_SHIFT)` function that takes a list of dictionaries as input and the list of shifts and that a new unique dictionary, where the numerical keys have been shifted by the numbers in `MAX_SHIFT`

In [115]:
# Write your code here
def shift_keys(dicts, MAX_SHIFT):
    counters = list(dicts)
    new_dict = {}
    for i in range(len(counters)):
        for key in counters[i].keys():
            new_dict[key + MAX_SHIFT[i]] = counters[i][key]
    
    return new_dict

In [116]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

[Counter({432: 0.14285714285714285,
          437: 0.07142857142857142,
          309: 0.07142857142857142,
          86: 0.07142857142857142,
          331: 0.07142857142857142,
          97: 0.07142857142857142,
          100: 0.07142857142857142,
          32: 0.07142857142857142,
          332: 0.07142857142857142,
          233: 0.07142857142857142,
          310: 0.07142857142857142,
          31: 0.07142857142857142,
          442: 0.07142857142857142}),
 Counter({6: 0.07692307692307693,
          765: 0.07692307692307693,
          224: 0.07692307692307693,
          203: 0.07692307692307693,
          557: 0.07692307692307693,
          176: 0.07692307692307693,
          590: 0.07692307692307693,
          711: 0.07692307692307693,
          527: 0.07692307692307693,
          757: 0.07692307692307693,
          919: 0.07692307692307693,
          57: 0.07692307692307693,
          685: 0.07692307692307693}),
 Counter({848: 0.08333333333333333,
          617: 0.08333333333333

In [39]:
shift_keys(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)), MAX_SHIFT)

{432: 0.14285714285714285,
 437: 0.07142857142857142,
 309: 0.07142857142857142,
 86: 0.07142857142857142,
 331: 0.07142857142857142,
 97: 0.07142857142857142,
 100: 0.07142857142857142,
 32: 0.07142857142857142,
 332: 0.07142857142857142,
 233: 0.07142857142857142,
 310: 0.07142857142857142,
 31: 0.07142857142857142,
 442: 0.07142857142857142,
 527: 0.07692307692307693,
 1286: 0.07692307692307693,
 745: 0.07692307692307693,
 724: 0.07692307692307693,
 1078: 0.07692307692307693,
 697: 0.07692307692307693,
 1111: 0.07692307692307693,
 1232: 0.07692307692307693,
 1048: 0.07692307692307693,
 1278: 0.07692307692307693,
 1440: 0.07692307692307693,
 578: 0.07692307692307693,
 1206: 0.07692307692307693,
 2400: 0.08333333333333333,
 2169: 0.08333333333333333,
 2020: 0.08333333333333333,
 2008: 0.08333333333333333,
 2425: 0.08333333333333333,
 2548: 0.08333333333333333,
 1839: 0.08333333333333333,
 1562: 0.08333333333333333,
 2369: 0.08333333333333333,
 2226: 0.08333333333333333,
 2512: 0.08333

Finally, we assemble all these utilities in a function

In [121]:
def build_freq_dict(sentence, MAXES=MAXES, MAX_SHIFT=MAX_SHIFT):
    hngrams = hash_ngrams(all_ngrams(sentence), MAXES)
    fhcodes = map(calc_rel_freq, hngrams)
    return shift_keys(fhcodes, MAX_SHIFT)

In [122]:
build_freq_dict('try something.')

{432: 0.14285714285714285,
 437: 0.07142857142857142,
 309: 0.07142857142857142,
 86: 0.07142857142857142,
 331: 0.07142857142857142,
 97: 0.07142857142857142,
 100: 0.07142857142857142,
 32: 0.07142857142857142,
 332: 0.07142857142857142,
 233: 0.07142857142857142,
 310: 0.07142857142857142,
 31: 0.07142857142857142,
 442: 0.07142857142857142,
 527: 0.07692307692307693,
 1286: 0.07692307692307693,
 745: 0.07692307692307693,
 724: 0.07692307692307693,
 1078: 0.07692307692307693,
 697: 0.07692307692307693,
 1111: 0.07692307692307693,
 1232: 0.07692307692307693,
 1048: 0.07692307692307693,
 1278: 0.07692307692307693,
 1440: 0.07692307692307693,
 578: 0.07692307692307693,
 1206: 0.07692307692307693,
 2400: 0.08333333333333333,
 2169: 0.08333333333333333,
 2020: 0.08333333333333333,
 2008: 0.08333333333333333,
 2425: 0.08333333333333333,
 2548: 0.08333333333333333,
 1839: 0.08333333333333333,
 1562: 0.08333333333333333,
 2369: 0.08333333333333333,
 2226: 0.08333333333333333,
 2512: 0.08333

## Converting the Dataset
We can now enrich the dataset with a numerical representation of the sentence. We use the utility functions and we call this new version: `dataset_num`

In [123]:
dataset[:2]

[('2478946', 'dan', 'Min bror taler for meget.'),
 ('1999574', 'dan', 'Han er meget nysgerrig.')]

In [124]:
dataset_num = []
for datapoint in tqdm(dataset):
    dataset_num += [list(datapoint) + [build_freq_dict(datapoint[2])]]

  0%|          | 0/300000 [00:00<?, ?it/s]

100%|██████████| 300000/300000 [00:44<00:00, 6731.78it/s]


In [125]:
dataset_num[:2]

[['2478946',
  'dan',
  'Min bror taler for meget.',
  {100: 0.08,
   233: 0.04,
   310: 0.04,
   86: 0.16,
   25: 0.04,
   437: 0.16,
   97: 0.08,
   432: 0.08,
   234: 0.04,
   15: 0.04,
   32: 0.12,
   327: 0.04,
   31: 0.04,
   442: 0.04,
   1294: 0.041666666666666664,
   1440: 0.041666666666666664,
   1222: 0.041666666666666664,
   1360: 0.041666666666666664,
   828: 0.041666666666666664,
   1356: 0.041666666666666664,
   1207: 0.08333333333333333,
   555: 0.125,
   860: 0.041666666666666664,
   1124: 0.041666666666666664,
   640: 0.041666666666666664,
   739: 0.041666666666666664,
   1066: 0.041666666666666664,
   1547: 0.041666666666666664,
   819: 0.041666666666666664,
   1466: 0.041666666666666664,
   1111: 0.041666666666666664,
   677: 0.041666666666666664,
   994: 0.041666666666666664,
   1232: 0.041666666666666664,
   770: 0.041666666666666664,
   1829: 0.043478260869565216,
   2032: 0.043478260869565216,
   1786: 0.043478260869565216,
   1656: 0.043478260869565216,
   2561

## Programming: Building ${X}$

You will now build the ${X}$ matrix.

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

Note that we used characters and not codes to make it more legible.

To create the ${X}$ matrix, we need to transform the dictionaries of `dataset_num` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the hash code frequency dictionaries from `dataset_num` corresponding to its 3rd index;
2. Convert the list of dictionaries into an ${X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the $n$-grams. Each item in this list will be a dictionary. You will call it `X_cat`

In [128]:
# Write your code here
X_cat = list(map(lambda x: x[3], dataset_num))

In [129]:
X_cat[0]

{100: 0.08,
 233: 0.04,
 310: 0.04,
 86: 0.16,
 25: 0.04,
 437: 0.16,
 97: 0.08,
 432: 0.08,
 234: 0.04,
 15: 0.04,
 32: 0.12,
 327: 0.04,
 31: 0.04,
 442: 0.04,
 1294: 0.041666666666666664,
 1440: 0.041666666666666664,
 1222: 0.041666666666666664,
 1360: 0.041666666666666664,
 828: 0.041666666666666664,
 1356: 0.041666666666666664,
 1207: 0.08333333333333333,
 555: 0.125,
 860: 0.041666666666666664,
 1124: 0.041666666666666664,
 640: 0.041666666666666664,
 739: 0.041666666666666664,
 1066: 0.041666666666666664,
 1547: 0.041666666666666664,
 819: 0.041666666666666664,
 1466: 0.041666666666666664,
 1111: 0.041666666666666664,
 677: 0.041666666666666664,
 994: 0.041666666666666664,
 1232: 0.041666666666666664,
 770: 0.041666666666666664,
 1829: 0.043478260869565216,
 2032: 0.043478260869565216,
 1786: 0.043478260869565216,
 1656: 0.043478260869565216,
 2561: 0.043478260869565216,
 1686: 0.043478260869565216,
 2372: 0.08695652173913043,
 1683: 0.043478260869565216,
 2162: 0.04347826086956

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html. You will set the `sparse` argument to False. Call the result `X`.

In [180]:
# Write your code here
v = DictVectorizer(sparse=False)
X = v.fit_transform(X_cat)

In [181]:
X.shape

(300000, 2583)

In [182]:
X[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [183]:
# Write your code here
y_cat = list(map(lambda x: x[1], dataset_num))

In [184]:
y_cat[:5]

['dan', 'dan', 'cmn', 'fra', 'swe']

Extract the set of language symbols and name it `y_symbols`. Then build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2idx`and `idx2lang`. Such a conversion is not necessary with sklearn. We do it because many other many machine-learning toolkits (keras or pytorch) require a numerical $\mathbf{y}$ vector and to learn how to carry out this conversion.

In [185]:
# Write your code here
y_symbols = set(y_cat)
idx2lang = {i:list(y_symbols)[i] for i in range(len(y_symbols))}
lang2idx = {list(y_symbols)[i]:i for i in range(len(y_symbols))}


In [186]:
idx2lang

{0: 'cmn', 1: 'eng', 2: 'swe', 3: 'jpn', 4: 'fra', 5: 'dan'}

In [187]:
lang2idx

{'cmn': 0, 'eng': 1, 'swe': 2, 'jpn': 3, 'fra': 4, 'dan': 5}

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [188]:
# Write your code here
y = [lang2idx[lang] for lang in y_cat]

In [189]:
y[:5]

[5, 5, 0, 4, 2]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call your classifier `clf`

In [190]:
# Write your code here
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True)

In [191]:
clf

MLPClassifier(hidden_layer_sizes=(50,), max_iter=5, verbose=True)

### Training and Validation Sets

You will now split the dataset into a training and validation sets

#### We split the dataset
We use a training set of 80% and a validation set of 20%

In [192]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [193]:
# Write your code here
model = clf.fit(X_train, y_train)

Iteration 1, loss = 0.29983999
Iteration 2, loss = 0.04097059
Iteration 3, loss = 0.02924281
Iteration 4, loss = 0.02421369
Iteration 5, loss = 0.02124722


/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [194]:
# Write your code here
y_val_pred = model.predict(X_val)

In [195]:
y_val_pred[:20]

array([4, 3, 5, 5, 5, 5, 1, 1, 3, 3, 5, 0, 3, 2, 2, 0, 5, 1, 0, 0])

In [196]:
y_val[:20]

[4, 3, 5, 5, 5, 5, 1, 1, 3, 3, 5, 0, 3, 2, 2, 0, 5, 1, 0, 0]

#### Evaluating

Use the `accuracy_score()` function to evaluate your model on the validation set

In [197]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

0.9915

In [198]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         cmn       1.00      1.00      1.00     10015
         eng       1.00      0.99      1.00     10023
         swe       0.98      0.98      0.98     10015
         jpn       1.00      1.00      1.00      9998
         fra       1.00      1.00      1.00      9957
         dan       0.98      0.98      0.98      9992

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.9915
Macro F1 0.9915056013388895


### Confusion Matrix

In [199]:
confusion_matrix(y_val, y_val_pred)

array([[9994,    1,    5,    9,    0,    6],
       [   0, 9972,   10,    0,   20,   21],
       [   0,   12, 9786,    0,    7,  210],
       [  18,    0,    0, 9980,    0,    0],
       [   0,   12,    7,    1, 9926,   11],
       [   0,   16,  140,    0,    4, 9832]])

You may try to increase the number of iterations to improve the score. You may also try change the parameters of the multilayer percetron.

## Predict the language of a text

Now you will predict the languages of the strings below.

In [200]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

In [201]:
build_freq_dict('Salut les gars !')

{331: 0.1875,
 234: 0.125,
 15: 0.125,
 69: 0.0625,
 432: 0.0625,
 86: 0.1875,
 32: 0.0625,
 31: 0.0625,
 437: 0.0625,
 333: 0.0625,
 1078: 0.06666666666666667,
 640: 0.06666666666666667,
 582: 0.06666666666666667,
 1542: 0.06666666666666667,
 1492: 0.06666666666666667,
 900: 0.06666666666666667,
 739: 0.06666666666666667,
 1319: 0.06666666666666667,
 1238: 0.13333333333333333,
 982: 0.06666666666666667,
 1415: 0.06666666666666667,
 557: 0.06666666666666667,
 1161: 0.06666666666666667,
 1020: 0.06666666666666667,
 1803: 0.07142857142857142,
 1608: 0.07142857142857142,
 2199: 0.07142857142857142,
 2349: 0.07142857142857142,
 2284: 0.07142857142857142,
 1958: 0.07142857142857142,
 1720: 0.07142857142857142,
 1925: 0.07142857142857142,
 2370: 0.07142857142857142,
 2546: 0.07142857142857142,
 1752: 0.07142857142857142,
 1805: 0.07142857142857142,
 1670: 0.07142857142857142,
 2269: 0.07142857142857142}

Create features vectors from this list. Call this matrix `X_test`

In [202]:
# Write your code here

X_test = v.transform(list(map(build_freq_dict, docs)))
X_test.shape

(4, 2583)

In [203]:
X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

And run the prediction that you will store in a variable called `pred_languages`

In [212]:
# Write your code here
pred_indexes = model.predict(X_test)
pred_languages=[]
for i in pred_indexes:
  pred_languages.append(idx2lang.get(i))

In [213]:
pred_languages

['fra', 'swe', 'eng', 'dan']

## Building the Model with PyTorch
You will now recreate a PyTorch model with the same architecture as in sklearn.

### The Model
Create a model identical to the one you created with sklearn. Use the same activation function for the hidden layer and no activation in the last layer.

In [215]:
len(langs)

6

In [220]:
# Write your code here
class Model(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 50)
        self.fc2 = nn.Linear(50, 6)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x        

In [221]:
input_dim = X.shape[1]
model = Model(input_dim)
model

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Write the loss `loss_fn` and optimizer `optimizer`. As optimizer, use the same as in sklearn. See here: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [222]:
# Write your code here. (The solution is given)
loss_fn = nn.CrossEntropyLoss()    # cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### The data loader

We convert the data to tensors

In [223]:
X_train = torch.Tensor(X_train)
y_train = torch.LongTensor(y_train)

X_val = torch.Tensor(X_val)
y_val = torch.LongTensor(y_val)

X_test = torch.Tensor(X_test)

In [224]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [225]:
model.train()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Fit your network on your training set. Write a code similar to that seen during the lecture and use five epochs to start with.

In [227]:
# Write your code here
for epoch in range(5):
    loss_train = 0
    for X_batch, y_batch in dataloader:
        y_batch_pred = model(X_batch)
        loss = loss_fn(y_batch_pred, y_batch)
        loss_train += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss_train)

754.7269158815034
171.3884837192163
137.00409126999148
116.79331097268914
102.47917095350022


Predict the validation set `X_val` in the form of logits. Call the result: `Y_val_pred_logits`

In [228]:
model.eval()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

In [233]:
# Write your code here
Y_val_pred_logits = model(X_val)

In [234]:
Y_val_pred_logits[:5]

tensor([[ -5.3669,  -6.3784,  -4.8668,  -8.3191,   8.3474,  -2.4181],
        [  0.3007,  -9.6264, -13.6366,  17.5873, -13.2007, -13.5541],
        [ -5.8057,  -5.9017,  -1.6422,  -8.2674, -12.8995,   7.8699],
        [ -5.3768,  -9.1691,  -8.0390,  -6.8934, -10.6851,  11.4169],
        [ -4.6127, -10.8675,  -8.6655,  -6.1467,  -8.3328,  10.8600]],
       grad_fn=<SliceBackward0>)

Predict the validation set `X_val` in the form of probabilities. Use `torch.softmax()` for that and call the result: `Y_val_pred_proba`

In [240]:
# Write your code here
Y_val_pred_proba = F.softmax(Y_val_pred_logits, dim=-1)

In [241]:
Y_val_pred_proba[:4]

tensor([[1.1065e-06, 4.0239e-07, 1.8244e-06, 5.7784e-08, 9.9998e-01, 2.1114e-05],
        [3.1083e-08, 1.5179e-12, 2.7518e-14, 1.0000e+00, 4.2552e-14, 2.9885e-14],
        [1.1501e-06, 1.0448e-06, 7.3948e-05, 9.8092e-08, 9.5484e-10, 9.9992e-01],
        [5.0885e-08, 1.1471e-09, 3.5518e-09, 1.1167e-08, 2.5190e-10, 1.0000e+00]],
       grad_fn=<SliceBackward0>)

Extract the categories from the probabilities in `Y_val_pred_proba`. Use the `torch.argmax()` function. Call the result `y_val_pred`. Check that the prediction corresponds to the real values.

In [242]:
# Write your code here
y_val_pred = torch.argmax(Y_val_pred_proba, dim=-1)

In [243]:
y_val_pred[:20]

tensor([4, 3, 5, 5, 5, 5, 1, 1, 3, 3, 5, 0, 3, 2, 2, 0, 5, 1, 0, 0])

In [244]:
y_val[:20]

tensor([4, 3, 5, 5, 5, 5, 1, 1, 3, 3, 5, 0, 3, 2, 2, 0, 5, 1, 0, 0])

Print the evaluation

In [245]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         cmn       1.00      1.00      1.00     10015
         eng       1.00      1.00      1.00     10023
         swe       0.98      0.98      0.98     10015
         jpn       1.00      1.00      1.00      9998
         fra       1.00      1.00      1.00      9957
         dan       0.98      0.98      0.98      9992

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.99205
Macro F1 0.9920512242690837


Print the confusion matrix

In [246]:
confusion_matrix(y_val, y_val_pred)

array([[9993,    1,    4,   14,    0,    3],
       [   1, 9980,   10,    0,   19,   13],
       [   0,   14, 9857,    0,   10,  134],
       [  19,    0,    0, 9979,    0,    0],
       [   0,   15,    5,    1, 9933,    3],
       [   0,   16,  189,    0,    6, 9781]])

Predict your languages with PyTorch. Reuse `X_test` and call the result `Y_test_pred_proba`.

In [247]:
# Write your code here
Y_test_pred_proba = F.softmax(model(X_test), dim=-1)

In [248]:
Y_test_pred_proba

tensor([[7.4468e-08, 2.6699e-07, 1.5333e-06, 1.5677e-08, 1.0000e+00, 2.4138e-07],
        [8.4458e-05, 9.4432e-06, 9.9937e-01, 1.7635e-06, 2.9352e-05, 5.0283e-04],
        [8.5775e-03, 9.1771e-01, 2.2886e-04, 3.0816e-04, 1.8217e-02, 5.4962e-02],
        [9.2468e-05, 5.4003e-07, 2.2075e-03, 2.3271e-06, 2.0853e-05, 9.9768e-01]],
       grad_fn=<SoftmaxBackward0>)

From the probabilities, extract the predicted languages and map them to strings. Call the results `pred_languages_pytorch`.

In [255]:
# Write your code here
pred_languages_pytorch = [idx2lang[idx]for idx in torch.argmax(Y_test_pred_proba, dim=-1).tolist()]

In [256]:
pred_languages_pytorch

['fra', 'swe', 'eng', 'dan']

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to:
   * Summarize CLD3 and outline its architecture
   * Identify the features used by CLD3
   * Describe your architecture and tell how it is different from CLD3
   * Include the feature matrix you computed manually
   * Outline the differences between sklearn and PyTorch

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 6, 2023.

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote in 2019 for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.